In [1]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)

Windows-10-10.0.19045-SP0
Python 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


In [2]:
class cfg:
    seed = 42
    nfolds = 5
    njobs = 2

In [3]:
import numpy as np
import pandas as pd
import os

import lazypredict
from lazypredict.Supervised import LazyRegressor #Regression

from sklearn.model_selection import train_test_split

In [4]:
# load data
train = pd.read_csv('../data/final/train.csv')
test = pd.read_csv('../data/final/test.csv')

train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,IsSynthetic,...,Monterey County,Napa County,Orange County,Other,Riverside County,San Francisco County,Santa Barbara County,Santa Clara County,Ventura County,Yolo County
0,2.39,15.00,3.83,1.11,1280.00,2.49,34.60,-120.12,0.98,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,3.72,17.00,6.01,1.05,1504.00,3.81,38.69,-121.22,0.95,1.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
2,4.78,27.00,6.54,1.10,1061.00,2.46,34.71,-120.45,1.58,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
3,2.41,16.00,3.35,0.97,1255.00,2.09,32.66,-117.09,1.34,1.00,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
4,3.75,52.00,4.28,1.07,1793.00,1.60,37.80,-122.41,4.50,1.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00


In [5]:
# quick info
TARGET = 'MedHouseVal'
FEATURES = [c for c in train.columns if c not in [TARGET]]

print(f'Target: {TARGET}\nFeatures: {FEATURES}')
print('Train set shape:', train.shape)
print('Test set shape:', test.shape)

Target: MedHouseVal
Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'IsSynthetic', 'AnomalyScore', 'rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'pca_lon', 'pca_lat', 'Alameda County', 'Contra Costa County', 'Fresno County', 'Kern County', 'Los Angeles County', 'Monterey County', 'Napa County', 'Orange County', 'Other', 'Riverside County', 'San Francisco County', 'Santa Barbara County', 'Santa Clara County', 'Ventura County', 'Yolo County']
Train set shape: (57777, 33)
Test set shape: (24759, 32)


In [6]:
# sample data
smp = train.sample(10000, random_state=cfg.seed)

# split data
x, x_val, y, y_val = train_test_split(
    smp[FEATURES],
    smp[TARGET],
    test_size=0.25,
    random_state=cfg.seed
)

In [7]:
# select regressors
list(enumerate(n[0] for n in lazypredict.Supervised.REGRESSORS))
idx = [0, 1, 3, 4, 6, 7, 8, 12, 13, 14, 19, 23, 24 , 25, 26, 33, 35, 36, 37, 40, 41]
mask = [True if est in idx else False for est in range(len(lazypredict.Supervised.REGRESSORS))]

from itertools import compress
lazypredict.Supervised.REGRESSORS = list(compress(lazypredict.Supervised.REGRESSORS, mask))
lazypredict.Supervised.REGRESSORS

[('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
 ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
 ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
 ('DummyRegressor', sklearn.dummy.DummyRegressor),
 ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
 ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
 ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
 ('HistGradientBoostingRegressor',
  sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor),
 ('HuberRegressor', sklearn.linear_model._huber.HuberRegressor),
 ('KNeighborsRegressor', sklearn.neighbors._regression.KNeighborsRegressor),
 ('LassoCV', sklearn.linear_model._coordinate_descent.LassoCV),
 ('LinearRegression', sklearn.linear_model._base.LinearRegression),
 ('LinearSVR', sklearn.svm._classes.LinearSVR),
 ('MLPRegressor', sklearn.neural_network._multilayer_perceptron.MLPRe

In [8]:
# lazypredict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(x, x_val, y, y_val)

100%|██████████| 21/21 [00:51<00:00,  2.44s/it]


In [9]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
LGBMRegressor,0.78,0.78,0.53,0.90
HistGradientBoostingRegressor,0.78,0.78,0.53,1.33
ExtraTreesRegressor,0.78,0.78,0.53,4.52
XGBRegressor,0.77,0.77,0.54,2.68
RandomForestRegressor,0.76,0.76,0.55,17.35
NuSVR,0.74,0.74,0.58,4.07
SVR,0.74,0.74,0.58,5.29
BaggingRegressor,0.74,0.74,0.58,1.89
KNeighborsRegressor,0.70,0.70,0.62,0.23
